<a href="https://colab.research.google.com/github/HaydenTCole/CPTS315/blob/Assignment3/hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We first need to upload the file "ml-latest-small.zip" downloaded from Canvas
try:
    from google.colab import files
    uploaded = files.upload()
except ImportError as e:
    pass

In [ ]:
# This cell provides some useful functions for generating the utility matrix.
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
import os
import pandas as pd
import numpy as np
import zipfile
import urllib.request
import sys

def unzip(name):
    with zipfile.ZipFile(name, 'r') as data:
        data.extractall()

class ml100k:
    @staticmethod
    def load():
        name = 'ml-100k'
        unzip(f"{name}.zip")
        ratings_path = os.path.join(name, 'u.data')
        ratings = pd.read_csv(
            ratings_path,
            sep='\t',
            names=["userid", "itemid", "rating", "timestamp"],
        )
        ratings = ratings.sort_values(by=['userid', 'itemid']).reset_index(drop=True)
        ratings = ratings.drop(columns=['timestamp'])

        movies_columns = [
            'itemid', 'title', 'release date', 'video release date',
            'IMDb URL ', 'unknown', 'Action', 'Adventure', 'Animation',
            "Children's", 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
            'Film-Noir', 'Horror', 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
            'Thriller' , 'War' , 'Western' ,
        ]

        movies_path = os.path.join(name, 'u.item')
        movies = pd.read_csv(
            movies_path,
            sep='|',
            names=movies_columns,
            encoding='latin-1',
        )
        # drop non necessary columns. From the third to the last column
        todrop = list(range(2, len(movies.columns)))
        movies = movies.drop(movies.columns[todrop], axis=1)

        return ratings, movies

def ids_encoder(ratings):
    users = sorted(ratings['userid'].unique())
    items = sorted(ratings['itemid'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())
    return ratings, uencoder, iencoder

def ratings_matrix(ratings):
    # return a rating matrix based with filling values
    return csr_matrix(pd.crosstab(ratings.userid, ratings.itemid, ratings.rating, aggfunc=sum).fillna(0).values)

# Generate utility information
def generate_utility_matrix():
    ratings, movies = ml100k.load()
    ratings, uencoder, iencoder = ids_encoder(ratings)
    return ratings

def create_model(rating_matrix, metric):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm='brute')
    model.fit(rating_matrix)
    return model


def nearest_neighbors(rating_matrix):
    """
    :param rating_matrix : rating matrix of shape (nb_users, nb_items)
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """
    rating_matrix = np.array(rating_matrix)
    if rating_matrix.shape != (943, 1682):
         raise ValueError("Rating Matrix is not of the right shape")

    model = create_model(rating_matrix, metric='cosine')
    similarities, neighbors = model.kneighbors(rating_matrix)
    return 1 - similarities[:, 1:], neighbors[:, 1:]


def find_candidate_items(userid):
    """
    Find candidate items for an active user

    :param userid : active user
    :param neighbors : users similar to the active user
    :return candidates : top 30 of candidate items
    """
    ratings_ = generate_utility_matrix()
    R_ = ratings_matrix(ratings_)
    ar = R_.toarray()
    similarities_, neighbors_ = nearest_neighbors(ar)
    user_neighbors = neighbors_[userid]
    activities = ratings.loc[ratings_.userid.isin(user_neighbors)]
    # sort items in decreasing order of frequency
    frequency = activities.groupby('itemid')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.itemid
    active_items = ratings_.loc[ratings.userid == userid].itemid.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]

    return candidates

In [6]:
def generate_centered_rating_matrix(ratings):
    """
    Please fill up your implementation.
    The input of the function is the original rating matrix "ratings" generated by the function "generate_utility_matrix()".
    The ratings is a pandas dataframe.

    Since we have 943 users and 1682 movies in the sytem.

    The output matrix should be a numpy array with shape 943 * 1682.
    The (i,j) element for the matrix is the centered rating from user i (with userid i) of move j (with itemid j).
    All the missing ratings are filling up with value 0.
    For the centered rating matrix, for example if the original rating matrix is:
    [[ 4 0 0 5 1 0],
     [ 5 5 4 0 0 0],
     [ 0 0 3 0 0 1]]
    Then the centered rating matrix is:
    [[ 2/3 0 0 5/3 -7/3 0],
     [ 1/3 1/3 -2/3 0 0 0],
     [ 0 0 1 0 0 -1]]
     Refer to Slide 8 - 1.

     Note that: All the missing values remain to be 0. The mean of each row is calculated among all the positive ratings (active ratings).
    """
    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    centered_rating_matrix = generate_utility_matrix()
    for row in centered_rating_matrix:
          row_avg = np.mean(centered_rating_matrix, axis=1)
          for x in row:
            if x != 0:
              x = x - row_avg
    raise NotImplementedError()
    ###########################################################################
    #                            END OF YOUR CODE                             #
    ###########################################################################
    return center_rating_matrix

In [ ]:
#1. Test the centered rating matrix
ratings = generate_utility_matrix()
centered_rating = generate_centered_rating_matrix(ratings)
centered_rating = np.array(centered_rating)

r_0 =   [1.38970588 ,-0.61029412,  0.38970588, -0.61029412, -0.61029412 , 1.38970588, 0.38970588, -2.61029412 , 1.38970588 ,-0.61029412]
r_15 =  [0.67142857, 0.  ,       0.  ,       0.67142857, 0. ,        0., 0.67142857, 0.67142857 ,0.67142857 ,0. ]
r_345 = [ 0.  , 1.70984456, -0.29015544 , 0.70984456,  0. ,  0., -1.29015544 , 0.  ,  0. ,  0. ]

r_0_ = centered_rating[0][0:10]
r_15_ = centered_rating[15][0:10]
r_345_ = centered_rating[345][0:10]

r_test = [r_0,r_15,r_345]
r_pre = [r_0_,r_15_,r_345_]

err = 0
for i in range(len(r_test)):
  err += np.linalg.norm(r_test[i] - r_pre[i])

print (f" Testing error is : {err}.")
if err >= 0.01:
  print("DOUBLE CHECK YOUR CALCULATION!")
else:
  print("YOU PASSED THE TEST!")

In [3]:
#We can obtain the similarity matrix and neighbor matrix using the centered rating matrix.

##########################################################################
"""
centered_rating = generate_centered_rating_matrix(ratings)
similarities, neighbors = nearest_neighbors(centered_rating)
"""
##########################################################################

"""
The shape of similarities and neighbors are both 943 x 20.
It is easy to obtain the closed (most similar) k=20 users for a given user with user_neighbors = neighbors[userid]
For example user1_neighbors = neighbors[1] will return the first 20 closed users for the user with userid = 1

The returnning list contains all the similar user information (userid) in decreasing order of similarities.
Similary, similarities[userid] contains the similarities information between user and its neighbors.
"""

def predict(ratings, userid, itemid, similarities, neighbors):
    """
    predict what score userid would have given to itemid.

    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction
        - similarities: a matrix contains the consin distance between user and its nearest 20 neighbors
        - neighbors: a matrix contains the most 20 closed users for each user

    :return
        - r_hat : predicted rating of user userid on item itemid
    """

    #Obtain neighbors N_u and associated similarity distance s_{u,v}
    user_similarities = similarities[userid] #list with size 20
    user_neighbors = neighbors[userid] #list with size 20

    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    #Something you need to do here
    # 1. Get mean rating of user userid
    # 2. Find users who rated item 'itemid'
    # 3. Find similar users to 'userid' who rated item 'itemid' (userids contained among the users found by last step)
    # 4. Obtain similarity distance from user_similarities

   #centered_rating = generate_centered_rating_matrix(ratings)
    #similarities, neighbors = nearest_neighbors(centered_rating)
    sorted_neighbors = np.arsort(user_neighbors)[::-1]
    weighted_sum = 0
    total_similarity = 0
    for neighbor_index in sorted_neighbors:
      similarity = user_similarities[neighbor_index]
      neighbor = user_neighbors[neighbor_index]
      neighbor_rating = ratings[neighbor, itemid]
      weighted_sum += similarity * neighbor_rating
      total_similarity += similarity
      #np.insert(r_hat, neighbor_index, weighted_sum/total_similarity)
    r_hat = weighted_sum/total_similarity

    raise NotImplementedError()

    ###########################################################################
    #                            END OF YOUR CODE                             #
    ###########################################################################

    return r_hat

In [ ]:
#2. Test the prediction for user i and item j

ratings = generate_utility_matrix()
centered_rating = generate_centered_rating_matrix(ratings)
centered_rating = np.array(centered_rating)
similarities, neighbors = nearest_neighbors(centered_rating)

test_case = [[0,422],[0,567],[0,404],[0,432],[100,257],[100,299],[850,6],[850,281]]
test_rating = [3.9486011373999577, 3.7872382581245776, 3.5241429399046993, 4.185430215904166, 3.491834899395883, 2.7749381804874993, 4.042253725491653, 3.50294950957094]
pres = []
for i in test_case:
  r_hat =  predict(ratings, i[0], i[1], similarities, neighbors)
  pres.append(r_hat)

err = np.linalg.norm(np.array(pres) - np.array(test_rating))
print (f" Testing error is : {err}.")
if err >= 0.01:
  print("DOUBLE CHECK YOUR CALCULATION!")
else:
  print("YOU PASSED THE TEST!")

In [4]:
def user2userRecommendation(ratings, userid: int, N : int):
    """
    Recommend movies to user (id = userid)
    :param
        - userid : user id for which we want to make prediction
    :return
        - r_list : list of recommended itemids
    """
    # find candidate items for the active user
    # You need to recommend movies from the set of candidates
    # candidates is a list of all the ponential items that can be recommended to user (id=userid)
    candidates = find_candidate_items(userid)
    ##########################################################################
    #                     TODO: Implement this function                      #
    ##########################################################################
    centered_rating = generate_centered_rating_matrix(ratings)
    similarities, neighbors = nearest_neighbors(centered_rating)
    for y in candidates:
        item_rating = predict(ratings, userid, candidates[y], similarities, neighbors)
        np.insert(rating_list, y, item_rating)
    sorted_ratings = np.argsort(rating_list)
    sorted_movies = [candidates[i] for i in sorted_ratings]
    r_list = sorted_movies[slice(N)]

    raise NotImplementedError()
    ###########################################################################
    #                            END OF YOUR CODE                             #
    ###########################################################################

    return r_list

In [ ]:
#3. Test the recommended movie
ratings = generate_utility_matrix()
test_case = [0,100,850]
items_testing = [[317, 356, 473, 650, 654], [14, 257, 272, 312, 1015], [95, 97, 99, 172, 180]]
r_movies = []
for i in test_case:
  toplist = user2userRecommendation(ratings,i,5)
  r_movies.append(toplist)

for item in r_movies:
  if sorted(item) in items_testing:
    items_testing.remove(sorted(item))
if len(items_testing)==0:
  print("YOU PASSED THE TEST!")
else:
  print("DOUBLE CHECK YOUR CALCULATION!")